# RDF-   NLS -  Encyclopaedia Britannica

This notebook is going to create the RDF triples to generate our RDLIB GRAPH

For each postprocess edition dataframe that we got from **Merging_EB_Terms.ipynb** (e.g. results_eb_1_edition_dataframe, results_eb_2_edition_dataframe, etc) we are going to add the information from the dataframe that we got from **Metadata_EB.ipynb** (metadata_eb_dataframe). 

The idea is to have per edition dataframe (and also supplement dataframe), all the information (which currently is splitted across several dataframes) in one. 


This notebook will store the final dataframes in results_NLS directory, and their name schema will be **final_eb_< NUM_EDITION >_dataframe**.

Per entry in these new dataframes we will have the following columns (see an example of one entry of the first edition):

- MMSID:                                              
- editionTitle:                          First edition, 1771, Volume 1, A-B
- editor:                                                  Smellie, William
- editor_date:                                                   1740-1795
- genre:                                                       encyclopedia
- language:                                                             eng
- termsOfAddress:                                                       NaN
- numberOfPages:                                                        832
- physicalDescription:               3 v., 160 plates : ill. ; 26 cm. (4to)
- place:                                                         Edinburgh
- publisher:              Printed for A. Bell and C. Macfarquhar; and so...
- referencedBy:           [Alston, R.C.  Engl. language III, 560, ESTC T...
- shelfLocator:                                                        EB.1
- editionSubTitle:        Illustrated with one hundred and sixty copperp...
- volumeTitle:            Encyclopaedia Britannica; or, A dictionary of ...
- year:                                                                1771
- volumeId:                                                       144133901
- metsXML:                                               144133901-mets.xml
- permanentURL:                            https://digital.nls.uk/144133901
- publisherPersons:                     [C. Macfarquhar, Colin Macfarquhar]
- volumeNum:                                                              1
- letters:                                                              A-B
- part:                                                                   0
- editionNum:                                                             1
- supplementTitle:                                                         
- supplementSubTitle:                                                      
- supplementsTo:                                                         []
- numberOfVolumes:                                                        6
- term:                                                                  OR
- definition:             A NEW A D I C T I A A, the name of several riv...
- relatedTerms:                                                          []
- header:                                           EncyclopaediaBritannica
- startsAt:                                                              15
- endsAt:                                                                15
- numberOfTerms:                                                         22
- numberOfWords:                                                         54
- positionPage:                                                           0
- typeTerm:                                                         Article
- altoXML:                                  144133901/alto/188082904.34.xml

### Loading the necessary libraries

In [31]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as pl
from rdflib import Graph, ConjunctiveGraph, Namespace, Literal
from rdflib.plugins.sparql import prepareQuery

In [32]:
import networkx as nx
import matplotlib.pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

In [33]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

### Functions

In [10]:
def plot_resource(results):

    G = rdflib_to_networkx_multidigraph(results)
    pos = nx.spring_layout(G, scale=3)
    edge_labels = nx.get_edge_attributes(G, 'r')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    nx.draw(G, with_labels=True)
    plt.show()

### 1. Loading the graph

In [ ]:
g = Graph()
g.parse("../../results_NLS/edition1st.ttl", format="ttl") 

List all the resources with the property eb:editor

In [ ]:
eb = Namespace("https://w3id.org/eb#")

q1 = prepareQuery('''
  SELECT ?Edition WHERE { 
    ?Edition eb:editor ?FullName. 
  }
  ''',
  initNs = { "eb": eb}
)


for r in g.query(q1):
      print(r.Edition)

Same query but asking more information about the resources obtained. 

In [ ]:
q2 = prepareQuery('''
  SELECT ?Subject ?Editor WHERE { 
    ?Subject eb:editor ?Editor.
  } 
  ''',
  initNs = { "eb": eb}
)

for r in g.query(q2):
  print(r.Subject, r.Editor)

In [ ]:
q2a = prepareQuery('''
SELECT DISTINCT ?name
    WHERE {
     ?instance eb:editor ?Editor.
     ?Editor eb:name ?name .
    }
    ''',
  initNs = { "eb": eb}
)

for r in g.query(q2a):
      print(r.name)

In [ ]:
res=g.query(q2a)
a=list(res)[0]
a.name

Same query asking for the first 10 resources with the properity eb.name

In [ ]:
q2 = prepareQuery('''
  SELECT ?Subject ?Name WHERE { 
    ?Subject eb:name ?Name.
  } 
  LIMIT 10
  ''',
  initNs = { "eb": eb}
)

for r in g.query(q2):
    print(r.Subject, r.Name)

In [ ]:
from rdflib import XSD
q2 = prepareQuery('''
SELECT DISTINCT ?name
WHERE {
 ?instance eb:editor ?editor;
           eb:place ?place;
           eb:title ?name.
  
}
LIMIT 10
  ''',
  initNs = { "eb": eb}
)

for r in g.query(q2, initBindings = {'?place' : Literal('London', datatype=XSD.string)}):
    print(r.name)

Asking for resources which name is "Smellie, Willian"

In [ ]:
from rdflib import XSD
q3 = prepareQuery('''
  SELECT ?Subject WHERE { 
    ?Subject eb:name ?Family.
  } 
  ''',
    initNs = { "eb": eb}
)

for r in g.query(q3, initBindings = {'?Family' : Literal('Smellie, William', datatype=XSD.string)}):
  print(r.Subject)

Asking for resources with name is ABACUS

In [ ]:
from rdflib import XSD
q3 = prepareQuery('''
  SELECT ?Subject WHERE { 
    ?Subject eb:name ?Term.
  } 
  ''',
    initNs = { "eb": eb}
)

for r in g.query(q3, initBindings = {'?Term' : Literal('ABACUS', datatype=XSD.string)}):
  print(r.Subject)

In [ ]:
from rdflib import XSD
q4 = prepareQuery('''
  SELECT ?Subject ?Term WHERE { 
    ?Subject eb:name ?Term.
  } 
  ''',
    initNs = { "eb": eb}
)

for r in g.query(q4, initBindings = {'?Term' : Literal('ABACUS', datatype=XSD.string)}):
  print(r.Subject, r.Term)

In [ ]:
q5 = prepareQuery('''
    SELECT ?edition
    WHERE {
       ?edition rdf:type eb:Edition .
    }
    ''',
  initNs = { "eb": eb}
)

for r in g.query(q5):
  print(r.edition)

In [ ]:
q5 = prepareQuery('''
    SELECT ?topic
    WHERE {
       ?topic rdf:type eb:Topic .
    }
    LIMIT 10
    ''',
  initNs = { "eb": eb}
)

for r in g.query(q5):
  print(r.topic)

In [ ]:
q6 = prepareQuery('''
    SELECT *
    WHERE {
       ?topic a eb:Topic .
       ?topic eb:relatedTerms ?relatedTerms . 
       ?topic eb:name ?name .
       ?topic eb:startsAt ?startsAt.
       ?topic eb:endsAt ?endsAt.
    }
    LIMIT 10
    ''',
  initNs = { "eb": eb}
)

for r in g.query(q6):
    print(r.topic, r.relatedTerms, r.name, r.startsAt, r.endsAt)

In [ ]:
q6 = prepareQuery('''
    SELECT *
    WHERE {
       ?article a eb:Article .
       ?article eb:relatedTerms ?relatedTerms . 
       ?article eb:name ?name .
       ?article eb:definition ?definition.
    }
    LIMIT 5
    ''',
  initNs = { "eb": eb}
)

for r in g.query(q6):
  print(r.article, r.relatedTerms, r.name, r.definition)

In [ ]:
q6 = prepareQuery('''
    SELECT *
    WHERE {
       ?article a eb:Article . 
       ?article eb:name ?name .
       ?article eb:definition ?definition.
    }
    LIMIT 5
    ''',
  initNs = { "eb": eb}
)

for r in g.query(q6):
  print(r.article, r.name, r.definition)

In [ ]:
q7 = prepareQuery('''
  SELECT  ?Subject ?Definition ?HasPart ?Title WHERE {
    ?Subject eb:name ?Term.
    ?Subject eb:definition ?Definition.
    ?HasPart eb:hasPart ?Subject.
    ?HasPart eb:title ?Title
  }  
  ''',
  initNs = { "eb": eb}
)

for r in g.query(q7, initBindings = {'?Term' : Literal('ABACUS', datatype=XSD.string)}):
  print(r.Subject, r.Definition, r.HasPart, r.Title)
  print("----")



In [ ]:
q7 = prepareQuery('''
  SELECT  ?Subject ?RelatedTerms WHERE {
    ?Subject eb:name ?Term.
    ?RelatedTerms eb:relatedTerms ?Subject.

    
  }  
  ''',
  initNs = { "eb": eb}
)

for r in g.query(q7, initBindings = {'?Term' : Literal('ABACUS', datatype=XSD.string)}):
  print(r.Subject, r.RelatedTerms)
  print("----")


In [ ]:
#G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
#pos = nx.spring_layout(G, scale=2)
#edge_labels = nx.get_edge_attributes(G, 'r')
#nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
#nx.draw(G, with_labels=True)

#if not in interactive mode for 
#plt.show()


### 3. Connecting with FUSEKI and using SPARQLWrapper

#### 3.1 Basic query

In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
    SELECT ?subject ?predicate ?object WHERE {   ?subject ?predicate ?object } LIMIT 5 
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

results

for result in results["results"]["bindings"]:
    print(result["subject"]["value"], result["predicate"]["value"], result["object"]["value"] )


#### 3.2 Describe query

In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
    DESCRIBE <https://w3id.org/eb/i/Article/9929192893804340_144850368_PAISLEY_0>
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()


##### 3.2.1 Plot Describe Result Graph 

In [ ]:

plot_resource(results)

##### 3.2.1 Print Describe Result Graph 

In [ ]:
for s,p,o in results:
      print(s,p,o)

In [78]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
term="ABACUS"
query="""
    PREFIX eb: <https://w3id.org/eb#>
    SELECT ?definition ?article ?spnum ?epnum ?year ?vnum ?enum ?rn ?permanentURL WHERE {
       ?article a eb:Article ;
                eb:name "%s" ;
                eb:definition ?definition ;
        OPTIONAL {?article eb:relatedTerms ?rt. 
                  ?rt eb:name ?rn. }
       ?e eb:hasPart ?v.
       ?v eb:number ?vnum.
       ?v eb:permanentURL ?permanentURL.
       ?v eb:hasPart ?article.
       ?e eb:publicationYear ?year.
       ?e eb:number ?enum.
       ?article eb:startsAtPage ?sp.
       ?sp   eb:number ?spnum .
       ?article eb:endsAtPage ?ep.
       ?ep eb:number ?epnum .
               
       }
    """ % (term)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r)
    break 

In [87]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
term="ABACUS"
query="""
PREFIX eb: <https://w3id.org/eb#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?definition ?a ?b  ?spnum ?epnum 
    WHERE {{
    	?b a eb:Article .
    	?b eb:name ?a .
        ?b eb:name "%s" .
        ?b eb:definition ?definition . 
        OPTIONAL {?b eb:relatedTerms ?rt. 
                  ?rt eb:name ?rn. }
       ?e eb:hasPart ?v.
       ?v eb:number ?vnum.
       ?v eb:permanentURL ?permanentURL.
       ?v eb:hasPart ?b.
       ?e eb:publicationYear ?year.
       ?e eb:number ?enum.
       ?b eb:startsAtPage ?sp.
       ?sp   eb:number ?spnum .
       ?b eb:endsAtPage ?ep.
       ?ep eb:number ?epnum .
        
        }
  		UNION {
    	?b a eb:Topic .
    	?b eb:name ?a .
        ?b eb:name "%s" .
        ?b eb:definition ?definition 
        
        OPTIONAL {?b eb:relatedTerms ?rt. 
                  ?rt eb:name ?rn. }
        
        ?e eb:hasPart ?v.
        ?v eb:number ?vnum.
        ?v eb:permanentURL ?permanentURL.
        ?v eb:hasPart ?b.
        ?e eb:publicationYear ?year.
        ?e eb:number ?enum.
        ?b eb:startsAtPage ?sp.
        ?sp   eb:number ?spnum .
        ?b eb:endsAtPage ?ep.
        ?ep eb:number ?epnum .
        
        }
   }
""" %(term, term)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r["a"]["value"])
    print(r["definition"]["value"])

ABACUS
in architeflure, signifies the superior part or member of the capital of a column, and serves as a kind of crowning to both. It was originally intended to represent a square tile covering a basket. The form of the abacus is not the same in all orders: in the Tuscan, Doric, and Ionic, it‘is generally square; but in the Corinthian and Compofite, its four sides are arched ir Avards, and embellilhed in the middle withornament, as a rose or other flower, Scammozzi uses abacus for a concave moulding on the capital of the Tuscan pedefial; and Palladio calls the plinth above the echinus, or boultin, in the Tufean and Doric orders, by the same name. See plate I. fig. i. and
ABACUS
a table strewed over with dust or sand, upon which the ancient mathematicians drew their figures, It also signified a cupboard, or buffet.
ABACUS
in architedture, signifies the superior part or member of the capital of a column, and serves as a kind of crowning to both. It was originally intended to represent a

In [57]:
for result in results["results"]["bindings"]:
    print("URI: %s -- Name %s -- Related Term: %s -- Definition %s " %(result["article"]["value"], result["name"]["value"], result["relatedTerms"]["value"], result["definition"]["value"] ))
    print("----")

KeyError: 'article'

In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
    PREFIX eb: <https://w3id.org/eb#>
       SELECT * WHERE {
       ?article a eb:Article .
       ?article eb:relatedTerms ?relatedTerms . 
       ?article eb:name ?name .
       ?article eb:definition ?definition.
  	   FILTER regex(?name, "^ABACISCUS") 
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print("URI: %s -- Name %s -- Related Term: %s -- Definition %s " %(result["article"]["value"], result["name"]["value"], result["relatedTerms"]["value"], result["definition"]["value"] ))
    print("----")

In [65]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT * WHERE {
       ?article a eb:Article .
       ?article eb:name ?name .
       FILTER regex(?name, "^ABACISCUS")
    }
    ORDER BY ?name
    LIMIT 10
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    print("URI: %s -- Name %s " %(result["article"]["value"], result["name"]["value"]))
    print("----")

URI: https://w3id.org/eb/i/Article/992277653804341_144133901_ABACISCUS_0 -- Name ABACISCUS 
----
URI: https://w3id.org/eb/i/Article/9929192893804340_144850366_ABACISCUS_0 -- Name ABACISCUS 
----


In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
    DESCRIBE <https://w3id.org/eb/i/Article/992277653804341_144133901_ABACUS_0>
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
plot_resource(results)

In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
    PREFIX eb: <https://w3id.org/eb#>
    SELECT ?year WHERE {
       ?edition a eb:Edition .
       ?edition eb:publicationYear ?year 
    }

""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [ ]:
results["results"]["bindings"][0]["year"]["value"]


In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
         PREFIX eb: <https://w3id.org/eb#>
         SELECT DISTINCT ?name
         WHERE {
         ?instance eb:editor ?Editor.
          ?Editor eb:name ?name .
       }
      
    """)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["name"]["value"]

In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?definition WHERE {
       ?article a eb:Article .
       ?article eb:name ?name .
       ?article eb:definition ?definition .
       FILTER regex(?name, "^ABACUS")
    }
    LIMIT 1
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["definition"]["value"]

In [103]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT (COUNT (DISTINCT ?a) as ?count)
    WHERE {
    	?a ?b ?d
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]

'19771'

In [104]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT (COUNT ( ?a) as ?count)
    WHERE {
    	?a ?b ?d
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]

'175586'

In [71]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT DISTINCT ?a
    WHERE {{
    	?b a eb:Article .
    	?b eb:name ?a}
  		UNION {
    	?b a eb:Topic .
    	?b eb:name ?a}
}
LIMIT 10
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r["a"]["value"])

PAISLEY
PRIOR
ANTHEM
ODYSSEY
CONSUMPTION
FRATERNITY
MATTURANTS
BILLET
KOS
STOCKHOLM


In [133]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT  ?d
    WHERE {
    	?b a eb:Article .
    	?b eb:definition ?d
        FILTER (CONTAINS(?d, "Scotland")) 
        FILTER (CONTAINS(?d, "Glasgow")) 
        OPTIONAL{FILTER CONTAINS(?d, "Mull") }
} LIMIT 10

""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r["d"]["value"])
    print("----")

a town of Scotland, in the county of Renfrew, six miles west of Glasgow.
----
a county of Scotland, lying westward of Glasgow, and comprehending the countries of Lorn, Cowal, Knapdale, Kintyfe, together with the islands Mull, Jura, Ifla, foe. It gives the title of duke to the noble family of Campbell.
----
a parliament town of Scotland, in the county of Argyle, of which it is the capital, fituatedin Lochhn, forty five miles north-west of Glasgow : W. long. 5 0 , N. lat 36° 28-.
----
a town of Scotland, in the county of Clydefdale, situated on the river Cl^de, eleven miles south-east of Glasgow : W. long. 3 0 jo', N. lat. 55 0
----
a town of Scotland, in the county of Renfrew, six miles west of Glasgow.
----
a parliament town of Scotland, in the county of Argyle, of which it is the capital, fituatedin Lochfn, forty five miles north-west of Glasgow : W. long. 5 0 , N. lat 36° 28' .
----
a place where great numbers of fifli are The principal fiffieries for salmon, herring, mackrel, pilcha

In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?definition ?y WHERE {
       ?article a eb:Article ;
                eb:name "PAISLEY" ;
                eb:definition ?definition .
       ?e eb:hasPart / eb:hasPart ?article.
       ?e eb:publicationYear ?y
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r["definition"]["value"], r["y"]["value"])

In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?definition ?y  ?r WHERE {
       ?article a eb:Article ;
                eb:name "ABACUS" ;
                eb:definition ?definition ;
                OPTIONAL {?article eb:relatedTerms ?r.}
       ?e eb:hasPart / eb:hasPart ?article.
       ?e eb:publicationYear ?y
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print("DEF: %s ----|| --- Year: %s "% (r["definition"]["value"], r["y"]["value"]))
    print("---")

In [ ]:
term="ABACUS"
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?definition ?article  ?year WHERE {
       ?article a eb:Article .
       ?article eb:name "ABACUS"  .
       ?article eb:definition ?definition .
       ?e eb:hasPart / eb:hasPart ?article.
       ?e eb:publicationYear ?year

    }
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i["year"]["value"])

In [ ]:
term="ABACUS"
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?definition ?article  ?year WHERE {
       ?article a eb:Article .
       ?article eb:name "%s"  .
       ?article eb:definition ?definition .
       ?e eb:hasPart / eb:hasPart ?article.
       ?e eb:publicationYear ?year
    }
""" % (term)

print(query)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i["year"]["value"])

In [ ]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
uri="<https://w3id.org/eb/i/Edition/992277653804341>"
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?publicationYear ?num ?title ?subtitle ?printedAt ?physicalDescription ?mmsid ?shelfLocator ?numberOfVolumes  WHERE {
       %s eb:publicationYear ?publicationYear ;
          eb:number ?num;
          eb:title ?title;
          eb:subtitle ?subtitle ;
          eb:printedAt ?printedAt;
          eb:physicalDescription ?physicalDescription;
          eb:mmsid ?mmsid;
          eb:shelfLocator ?shelfLocator;
          eb:numberOfVolumes ?numberOfVolumes. 
          
               
}
""" % (uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i["publicationYear"]["value"])

In [84]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
uri="<https://w3id.org/eb/i/Edition/992277653804341>"
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT (COUNT (DISTINCT ?v) as ?count)
    WHERE {
        %s eb:hasPart ?v.
    	?v ?b ?c
}
""" % (uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]


'3'

In [7]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT * WHERE {
       ?e a eb:Edition .
    }
""" 

print(query)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results


PREFIX eb: <https://w3id.org/eb#>
SELECT * WHERE {
       ?e a eb:Edition .
    }



{'head': {'vars': ['e']},
 'results': {'bindings': [{'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/992277653804341'}},
   {'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/9929192893804340'}}]}}

In [8]:
results

{'head': {'vars': ['e']},
 'results': {'bindings': [{'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/992277653804341'}},
   {'e': {'type': 'uri',
     'value': 'https://w3id.org/eb/i/Edition/9929192893804340'}}]}}

In [4]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?enum ?e ?y WHERE {
       ?e a eb:Edition ;
            eb:number ?enum ;
            eb:publicationYear ?y.
               
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i["enum"]["value"], i["e"]["value"], i["y"]["value"])

1 https://w3id.org/eb/i/Edition/992277653804341 1771
1 https://w3id.org/eb/i/Edition/9929192893804340 1773


In [16]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
uri="<https://w3id.org/eb/i/Edition/992277653804341>"
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?v ?vnum ?part ?letters WHERE {
       %s eb:hasPart ?v .
       ?v eb:number ?vnum ; 
          eb:letters ?letters .
          OPTIONAL {?v eb:part ?part; }
     
               
} 
""" % (uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    if "part" in i:
        print(i["v"]["value"], i["vnum"]["value"],i["letters"]["value"], i["part"]["value"])
    else:
        print(i["v"]["value"], i["vnum"]["value"], i["letters"]["value"])

https://w3id.org/eb/i/Volume/992277653804341_144133902 2 C-L
https://w3id.org/eb/i/Volume/992277653804341_144133903 3 M-Z
https://w3id.org/eb/i/Volume/992277653804341_144133901 1 A-B


In [25]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
uri="<https://w3id.org/eb/i/Volume/992277653804341_144133901>"
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?num ?title ?part ?metsXML ?volumeId ?permanentURL ?numberOfPages ?letters WHERE {
       %s eb:number ?num ;
          eb:title ?title;
          eb:metsXML ?metsXML;
          eb:volumeId ?volumeId;
          eb:permanentURL ?permanentURL;
          eb:numberOfPages ?numberOfPages;
          eb:letters ?letters.
       OPTIONAL {%s eb:part ?part. }
      
               
}
""" % (uri, uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i)

{'num': {'type': 'literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'value': '1'}, 'title': {'type': 'literal', 'value': 'Encyclopaedia Britannica; or, A dictionary of arts and sciences, compiled upon a new plan'}, 'metsXML': {'type': 'literal', 'value': '144133901-mets.xml'}, 'volumeId': {'type': 'literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#int', 'value': '144133901'}, 'permanentURL': {'type': 'literal', 'value': 'https://digital.nls.uk/144133901'}, 'numberOfPages': {'type': 'literal', 'value': '832'}, 'letters': {'type': 'literal', 'value': 'A-B'}}


In [100]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
uri="<https://w3id.org/eb/i/Volume/992277653804341_144133901>"
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT (COUNT (?t) as ?count)
    WHERE {
        %s eb:hasPart ?t .
        ?t ?b ?c 
        
} 
""" % (uri)
      
            
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]


'2707'

In [102]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
uri="<https://w3id.org/eb/i/Volume/992277653804341_144133901>"
query="""
PREFIX eb: <https://w3id.org/eb#>
SELECT (COUNT (DISTINCT ?t) as ?count)
    WHERE {
        %s eb:hasPart ?t .
        ?t a eb:Article
        
} 
""" % (uri)
      
            
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results

{'head': {'vars': ['count']},
 'results': {'bindings': [{'count': {'type': 'literal',
     'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'value': '2505'}}]}}

In [111]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
uri="<https://w3id.org/eb/i/Volume/992277653804341_144133901>"
query="""
       PREFIX eb: <https://w3id.org/eb#>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       SELECT (count (DISTINCT ?a) as ?count)
       WHERE {
            %s eb:hasPart ?b .
            ?b a eb:Article .
            ?b eb:name ?a.
      }


""" % (uri)
            
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]

'2440'

In [81]:
clear_r=[]
for s,p,o in results.triples((None, None, None)):
    data={}
    data["subject"]=str(s)
    data["predicate"]=str(p)
    data["object"]=str(o)
    clear_r.append(data)
    
for i in clear_r:
    if "startsAtPage" in i["predicate"]:
        startsAtPage= i["object"]
    if "endsAtPage" in i["predicate"]:
        endsAtPage= i["object"]

if startsAtPage == endsAtPage:
    for i in clear_r:
        if "endsAtPage" in i["predicate"]:
            clear_r.remove(i)
        
for i in clear_r:
    print(i)
   

{'subject': 'https://w3id.org/eb/i/Article/992277653804341_144133903_ROSA_0', 'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'object': 'https://w3id.org/eb#Article'}
{'subject': 'https://w3id.org/eb/i/Article/992277653804341_144133903_ROSA_0', 'predicate': 'https://w3id.org/eb#definition', 'object': 'in botany, a genus of the icofandria polygamia clals. The petals are five ; the calix has five flelhy fegments; and the feeds are numerous, rough, and inferred into the interior side of the calix-. There are 14 species, five of them natives of Britain, viz. the eglanteria, orfweet briar; the fpinofiffima, or burner rose ; the arvenfis, or whiteflowered , dogs-rose; the villofa, or apple-rose ; and the canina, or red-flowered dogs-rose, or hip-tree.'}
{'subject': 'https://w3id.org/eb/i/Article/992277653804341_144133903_ROSA_0', 'predicate': 'https://w3id.org/eb#name', 'object': 'ROSA'}
{'subject': 'https://w3id.org/eb/i/Article/992277653804341_144133903_ROSA_0', 'predicate'

In [56]:
for s,p,o in results.triples((None, None, None)):
    a=s
    
a.n3()

'<https://w3id.org/eb/i/Edition/992277653804341>'

In [53]:
results.triples

<bound method ConjunctiveGraph.triples of <Graph identifier=Nf4bcb9045c244d0fb3903f5f315c191f (<class 'rdflib.graph.ConjunctiveGraph'>)>>

In [8]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
PREFIX eb: <https://w3id.org/eb#>
SELECT ?topic ?y WHERE {
       ?topic a eb:Topic ;
    }
LIMIT 10
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
for r in results["results"]["bindings"]:
    print(r["topic"]["value"])

https://w3id.org/eb/i/Topic/992277653804341_144133901_ALGEBRA_1
https://w3id.org/eb/i/Topic/9929192893804340_144850367_GHMMAR_0
https://w3id.org/eb/i/Topic/9929192893804340_144850368_RELIGIONTHEOLOGY_0
https://w3id.org/eb/i/Topic/9929192893804340_144850366_BREWING_1
https://w3id.org/eb/i/Topic/9929192893804340_144850367_LATV_0
https://w3id.org/eb/i/Topic/992277653804341_144133902_TACIV_0
https://w3id.org/eb/i/Topic/9929192893804340_144850367_DEB_0
https://w3id.org/eb/i/Topic/9929192893804340_144850367_L_0
https://w3id.org/eb/i/Topic/992277653804341_144133903_MECHANICS_0
https://w3id.org/eb/i/Topic/992277653804341_144133903_PNEUMATICS_0


In [28]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
uri="<https://w3id.org/eb/i/Volume/992277653804341_144133902>"
query="""
    PREFIX eb: <https://w3id.org/eb#>
    SELECT ?vnum ?letters ?part WHERE {
       %s eb:number ?vnum ;
          eb:letters ?letters .
           OPTIONAL {%s eb:part ?part; }
       
    
    } """ % (uri, uri)
sparql.setQuery(query)         
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
i=results["results"]["bindings"][0]


if "part" in i:
    data= i["vnum"]["value"]+ " "+ i["letters"]["value"]+ "Part "+ i["part"]["value"]
else:
    data= i["vnum"]["value"]+ " "+ i["letters"]["value"]

data

'2 C-L'

In [30]:
sparql = SPARQLWrapper("http://localhost:3030/edition1st/sparql")
sparql.setQuery("""
    DESCRIBE <https://w3id.org/eb/i/Volume/992277653804341_144133902>
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results

<Graph identifier=Naaf3cb1775434b5ab8e801dd71cf3b88 (<class 'rdflib.graph.ConjunctiveGraph'>)>